In [2]:
!pip install git+https://github.com/samoturk/mol2vec

  Cloning https://github.com/samoturk/mol2vec to /tmp/pip-req-build-ftkwl7vo
  Running command git clone --filter=blob:none --quiet https://github.com/samoturk/mol2vec /tmp/pip-req-build-ftkwl7vo
  Resolved https://github.com/samoturk/mol2vec to commit 850d944d5f48a58e26ed0264332b5741f72555aa
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 9.7 MB/s eta 0:00:000m eta 0:00:010:00:01
  Using cached seaborn-0.11.2-py3-none-any.whl (292 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 KB 7.4 MB/s eta 0:00:00
  Created wheel for mol2vec: filename=mol2vec-0.1-py3-none-any.whl size=14038 sha256=ceb95932fe94e1853efa36ce3132a97b6b220c912831d43abb5411d76d21135c
  Stored in directory: /tmp/pip-ephem-wheel-cache-nzn8yso2/wheels/28/66/2a/543e91280db70c489a72de9c9fa26dca4caa7113721fb03b86
Successfully built mol2vec


In [3]:
pip install -U gensim==3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 10.9 MB/s eta 0:00:00m eta 0:00:010:01:01
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.2.0-cp38-cp38-linux_x86_64.whl size=15446003 sha256=1cf5aceceb24098013c4f89173d47d09c0ceb6d8c594013a2e48ada2c356ff68
  Stored in directory: /home/and/.cache/pip/wheels/47/c1/60/53087a96fa4060b88cc6e286fe9c5374e93d774c78be97ab49
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 3.0.0
    Uninstalling gensim-3.0.0:
      Successfully uninstalled gensim-3.0.0
Note: you may need to restart the kernel to use updated packages.


In [44]:
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
from catboost import CatBoostClassifier

from rdkit import Chem
from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec

DATA_PATH = Path("../data")
SEED = 42
TEST_SIZE = 0.2

In [161]:
df = pd.read_csv(DATA_PATH / "base/train.csv").drop("Unnamed: 0", axis=1)
target = df.Active.astype(int)
df = df.drop("Active", axis=1)

In [162]:
df["mol"] = df['Smiles'].apply(lambda x: Chem.MolFromSmiles(x))

In [163]:
model = word2vec.Word2Vec.load('../models/model_300dim.pkl')

In [164]:
print('Molecular sentence:', mol2alt_sentence(df['mol'][1], radius=1))
print('\nMolSentence object:', MolSentence(mol2alt_sentence(df['mol'][1], radius=1)))
print('\nDfVec object:',DfVec(
    sentences2vec(MolSentence(mol2alt_sentence(df['mol'][1], radius=1)), model, unseen='UNK')))

Molecular sentence: ['2246728737', '3542456614', '2245384272', '1173125914', '2245384272', '2257970297', '2092489639', '1634606847', '2968968094', '2803848648', '2968968094', '2142032900', '2968968094', '2117068077', '2976033787', '3482873808', '3217380708', '970433791', '3218693969', '951226070', '3218693969', '98513984', '3218693969', '951226070', '3217380708', '2905660137', '864662311', '26234434', '3218693969', '994485099', '2968968094', '2811394787', '1016845826']

MolSentence object: MolSentence with 33 words

DfVec object: (33, 300) dimensional vector


In [165]:
#Constructing sentences
df['sentence'] = df.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
df['mol2vec'] = [DfVec(x).vec for x in sentences2vec(df['sentence'], model, unseen='UNK')]
# X = np.array([x.vec for x in df['mol2vec']])
# y = target.values


In [168]:
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=TEST_SIZE, random_state=SEED)

In [175]:
pred = cat_model.predict(test_df)

In [176]:
pred = cat_model.predict(test_df)
test_df = pd.read_csv(DATA_PATH / "base/test.csv")
test_df["Active"] = pred
test_df["Active"] = test_df["Active"].astype(int)

In [177]:
test_df.to_csv("submission.csv", index=False)